## Setup
General Idea Begin by importing all of the pathology reports into a list where each report is an index of the list. Hopefully all in text data Then Iterate through each pathology report scanning for text within the report, look for keywords such "Cancer" and Cancer Types (not sure, ask Alexis) Create lists to hold each cancer type folder we may need (not sure how many, possible 20-25) If that cancer type was found in a report, iterate through the list to create that directory titled "cancer_type" from the list. Then based on the cancer type found (maybe append cancer type to the end of each pathology report or at the 2D array index from the list) save each seperate pathology report to their respective directory based on cancer type.

No NLP or AI needed, just text extract for types of cancer we will be getting. Held in a list and checked for maybe. Check for edge cases and NOTS in the list

In [7]:
from os import listdir
from concurrent.futures import ThreadPoolExecutor, as_completed
from os.path import isfile, join
import shutil
import time
import os
import re
import hashlib

# Path to the directory containing pathology reports
pathologypath = "../NLPRawData/messages"
pathology_file_untouched = [] # Untouched files for saving
pathology_reports = [] # Store the filename and content

# Get the list of file names in the pathology directory
pathology_files = [f for f in listdir(pathologypath) if isfile(join(pathologypath, f))]

# Iterate over the files and read their content as a single string
for report in pathology_files:
    filepath = join(pathologypath, report)  # Construct the full file path
    with open(filepath, "r") as file:
        content = file.read()  # Read the entire file content into a string
        content = content.replace('\\X0D\\\\X0D\\', '\n\n') # Remove all weird new line characters
        content = content.replace('\\X0D\\', '\n')
        content = content.replace('.br', '') # Remove all line break characters
        content = content.replace('^', ' ') # Remvove other uneeded spacers
        content = content.replace('|', ' ')                                                                                                          
        content = re.sub("\\\\\\\\", "\n", content) # Substitute \\\\\\\ for a new line
        pathology_reports.append({"filename": report, "content": content}) # Append the cleaned document to the array

pathology_reports.sort(key=lambda x: int(os.path.splitext(x["filename"])[0])) # Sort the reports to appear in the same order as they are read.
# Sorted by lowest number first

# Print last 5 to check sorted order
for report in pathology_reports[-5:]:
    print(report["filename"])

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../NLPRawData/messages'

### Save reports by PHN or PID found within the report for better identifying cancer types

In [8]:
# Initialize the phn list
phn = []

# Define directories
phn_sorted_path = "../Sorted_by_PHN_Cleaned_Reports" #Path to hold sorted reports

# Remove existing files and directory if they exist
if os.path.exists(phn_sorted_path):  # Check if the directory exists
    shutil.rmtree(phn_sorted_path)  # Delete its contents if it does
    print("Reports Deleted")

# Create the directory for PID sorting
os.makedirs(phn_sorted_path, exist_ok=True)

# Parallely process all reports
def process_report(report):
    content = report["content"]  # Get the content of the report
    patient_id = ""  # Initialize patient_id variable
    
    if "PID" in content:  # Find PID in report, all should have them
        line = content.split("PID", 1)[1].strip()  # Split at PID and get text after it
        patient_id = line[3:13]  # Extract patient ID (3 to 13 characters after PID for most reports)

    # Check if the patient ID is valid (numeric and unique) and add it to phn list
    if patient_id.isnumeric() and patient_id not in phn:
        phn.append(patient_id)  # Append unique patient ID to phn
    elif patient_id not in phn: # Else check that we have the right ID
        patient_id = line[0:10] # ([0 - 10] for other reports if PID wasn't from [3 - 13])
        phn.append(patient_id) # Append if the PID has a different offset
    
    # Create the directory for the patient if it's in the phn list, otherwise move to 'unsorted'
    if patient_id in phn:
        # Ensure the directory for the patient exists
        patient_dir = os.path.join(phn_sorted_path, patient_id)
        os.makedirs(patient_dir, exist_ok=True)
        
        # Save the report
        filename = report.get("filename", "unknown.txt")  # Get filename (default to 'unknown.txt'), They should all have a filename
        file_path = os.path.join(patient_dir, filename)  # Full path to save the file
        
        with open(file_path, "w") as f:
             f.write(content)  # Write the content of the report to the file

# Set up the ThreadPoolExecutor to parallelize the processing of reports (done to speed it up)
with ThreadPoolExecutor() as executor:
    # Parallelize the report processing, Python is done with executor
    futures = [executor.submit(process_report, report) for report in pathology_reports]

print(f"Reports sorted by PHN into {phn_sorted_path}")




Reports Deleted
Reports sorted by PHN into ./Sorted_by_PHN_Cleaned_Reports


In [14]:
# Keywords to check for each cancer type, most common keywords and cancer types. Used for sorting into directorties
# To improve accuracy, you would need better keyword detection but it is difficult when I have a hard time reading the reports
breast_cancer_keywords = ["Breast Cancer", "Breast", "Left Breast", "Right Breast", "Invasive Ductal Carcinoma", "Invasive Lobular Carcinoma", "Ductal Carcinoma In Situ", "Lobular Carcinoma In Situ", "Estrogen Receptor Positive", "Progesterone Receptor Positive", "HER2 Positive", "Triple Negative", "Sentinel Lymph Node Biopsy", "Axillary Lymph Node Dissection", "Nipple", "HER2 Signals per Cell", "Invasive ductal"]
lung_cancer_keywords = ["Lung", "Lung Cancer", "TTF-1", "Non-Small Cell Lung Cancer", "Small Cell Lung Cancer", "Large Cell Carcinoma", "Mucinous Adenocarcinoma", "Bronchial Carcinoma", "Endobronchial Carcinoma", "Pulmonary Carcinoma", "Lung Metastasis", "Pleural Effusion", "Bronchoscopy", "Lung Biopsy", "TRANSBRONCHIAL BIOPSIES", "bronchovascular"]
lymph_cancer_keywords = ["Lymphoma", "Non-Hodgkin Lymphoma", "Hodgkin Lymphoma", "Lymph", "left supraclavicular lymph node","B-cell Lymphoma", "T-cell Lymphoma", "Mantle Cell Lymphoma", "Follicular Lymphoma", "Diffuse Large B-cell Lymphoma", "Burkitt Lymphoma", "MALT Lymphoma", "Lymph Node Biopsy", "Lymphadenopathy", "Extranodal Lymphoma", "lymphocytic"]
brain_cancer_keywords = ["Brain Cancer", "Brain Tumor", "Brain", "Glioblastoma", "Astrocytoma", "Meningioma", "Oligodendroglioma", "craniopharyngioma", "Medulloblastoma", "Ependymoma", "Brain Metastasis", "Craniotomy", "Neurosurgery", "Brain Biopsy", "Stereotactic Radiosurgery", "Intracranial Neoplasm"]
skin_cancer_keywords = ["Skin Cancer", "Skin", "Melanoma", "Melanoma In Situ" "Basal Cell Carcinoma", "Squamous Cell Carcinoma", "Nerve-Derived Tumor", "Merkel Cell Carcinoma", "Skin Lesion", "Skin Biopsy", "Wide Local Excision", "Mohs Surgery", "Sentinel Lymph Node Biopsy", "Dermatologic Surgery", "Skin Metastasis", "Superficial Spreading Melanoma", "Nodular Melanoma"]
colon_cancer_keywords = ["Colon Cancer", "Colon", "Colorectal Cancer", "Rectal Cancer", "RECTAL BIOPSY", "Colonic Mucosa", "Adenocarcinoma of Colon", "Colonic Polyp", "Colonoscopy", "Sigmoid Colon Cancer", "Ascending Colon Cancer", "Descending Colon Cancer", "Transverse Colon Cancer", "Colon Resection", "Colectomy", "Lynch Syndrome", "Familial Adenomatous Polyposis" ,"ADENOMATOUS MUCOSA"]
uterine_or_ovarian_cancer_keywords = ["Ovarian Cancer", "Ovaries", "Ovarian", "Omentectomy", "ovarian papillary", "fallopian tube","Uterine Cancer", "Uterine", "Endometrial Cancer", "Uterine Sarcoma", "Uterine Malignancy", "Endometrial Hyperplasia", "Uterine Biopsy", "Hysterectomy", "Myometrial Invasion", "Endometrial Sampling", "Uterine Bleeding", "Uterine Mass", "Endometrial Thickening", "Uterine Metastasis", "Lymphadenectomy", "Pelvic Washing"]
vulvar_cancer_keywords = ["Vulvar Cancer", "Vulvar", "Vulvar Intraepithelial Neoplasia", "Vulvar Lesion", "Vulvar Mass", "GROIN LESION", "Vulvar Biopsy", "Wide Local Excision", "Radical Vulvectomy", "Vulvar Resection", "Inguinal Lymph Node Dissection", "Vulvar Reconstruction", "Vulvar Metastasis", "HPV Related Vulvar Cancer", "Vulvar Melanoma", "introitus", "female genital tract"]
leukemia_cancer_keywords = ["Leukemia", "Acute Myeloid Leukemia", "Acute Lymphoblastic Leukemia", "Chronic Myeloid Leukemia", "Chronic Lymphocytic Leukemia", "Bone Marrow Biopsy", "Blast Cells", "Flow Cytometry", "Cytogenetics", "Philadelphia Chromosome","Bone Marrow", "Bone Marrow Transplant", "Stem Cell Transplant", "White Blood Cell Count", "Leukemic Cells"]
cervical_cancer_keywords = ["Cervical Cancer", "Cervical Intraepithelial Neoplasia", "Pap Smear", "Colposcopy", "Cervix", "Endocervical Curettings", "LEEP Procedure", "Cone Biopsy", "Radical Hysterectomy", "Trachelectomy", "Cervical Biopsy", "Cervical Metastasis", "Parametrial Invasion", "Cervical Screening"]
stomach_cancer_keywords = ["Stomach Cancer", "Gastric Cancer", "Gastric Adenocarcinoma", "Gastric Lymphoma", "Gastrectomy", "Endoscopy", "Gastric Biopsy", "Gastric Mass", "Gastric Ulcer", "Helicobacter Pylori", "Gastric Resection", "Stomach Metastasis", "Linitis Plastica", "Gastric Cardia Cancer"]
pancreatic_cancer_keywords = ["Pancreatic Cancer", "Pancreas", "Pancreatic Adenocarcinoma", "Pancreatic Neuroendocrine Tumor", "Whipple Procedure", "Pancreatic Mass", "Pancreatic Biopsy", "ERCP", "Pancreatic Duct Dilation", "Pancreatic Resection", "Distal Pancreatectomy", "Pancreatic Head Mass", "Pancreatic Tail Mass", "Pancreatic Metastasis", "CA 19-9"]
kidney_cancer_keywords = ["Kidney Cancer", "Kidney", "Renal Cell Carcinoma", "Clear Cell Carcinoma", "Papillary Renal Cell Carcinoma", "Chromophobe Renal Cell Carcinoma", "Nephrectomy", "Partial Nephrectomy", "Renal Mass", "Renal Biopsy", "Renal Tumor", "Kidney Metastasis", "Von Hippel-Lindau Disease", "Renal Cyst", "Renal Artery Embolization", "Renal"]
thyroid_cancer_keywords = ["Thyroid Cancer", "Thyroid", "Papillary Thyroid Cancer", "Follicular Thyroid Cancer", "Medullary Thyroid Cancer", "Anaplastic Thyroid Cancer", "Thyroidectomy", "Thyroid Nodule", "Fine Needle Aspiration", "Thyroid Ultrasound", "Thyroid Biopsy", "Radioactive Iodine", "Thyroid Metastasis", "Thyroglobulin", "Thyroid Lymph Node"]
liver_cancer_keywords = ["Liver Cancer", "Liver", "Liver nodule", "Liver Mass","Liver Biopsy","Hepatocellular Carcinoma", "HCC", "Cholangiocarcinoma", "Liver Metastasis", "Liver Mass", "Liver Biopsy", "Liver Resection", "Hepatectomy", "TACE", "Radiofrequency Ablation", "Portal Vein Thrombosis", "Cirrhosis", "Alpha-Fetoprotein"]
bladder_cancer_keywords = ["Bladder Cancer", "Bladder","Transitional Cell Carcinoma", "Urothelial Carcinoma", "TURBT", "Cystectomy", "Bladder Biopsy", "Bladder Mass", "Bladder Tumor", "Urine Cytology", "BCG Treatment", "Intravesical Therapy", "Bladder Metastasis", "Hematuria", "Urinary Diversion"]
# All cancer keywords held in dictionary
cancer_keywords = {
    "breast_cancer": breast_cancer_keywords,
    "lung_cancer": lung_cancer_keywords,
    "lymph_cancer": lymph_cancer_keywords,
    "brain_cancer": brain_cancer_keywords,
    "skin_cancer": skin_cancer_keywords,
    "colon_cancer": colon_cancer_keywords,
    "uterine_or_ovarian_cancer": uterine_or_ovarian_cancer_keywords,
    "vulvar_cancer": vulvar_cancer_keywords,
    "leukemia_cancer": leukemia_cancer_keywords,
    "cervical_cancer": cervical_cancer_keywords,
    "stomach_cancer": stomach_cancer_keywords,
    "pancreatic_cancer": pancreatic_cancer_keywords,
    "kidney_cancer": kidney_cancer_keywords,
    "thyroid_cancer": thyroid_cancer_keywords,
    "liver_cancer": liver_cancer_keywords,
    "bladder_cancer": bladder_cancer_keywords,
}
print("Done")

Done


In [16]:
from rapidfuzz import fuzz
phn_sorted_path = "./Sorted_by_PHN_Cleaned_Reports" #Path to hold sorted reports
# Read all newly sorted files by PHN
all_files = {}
for root, dirs, files in os.walk(phn_sorted_path):
    for file in files:
        sorted_file_path = os.path.join(root, file)
        # If the folder is not already in the dictionary, add it
        if root not in all_files:
            all_files[root] = {"content": "", "files": []}
        
        # Read the file content and append to the folder's combined content
        with open(sorted_file_path, "r") as f:
            content = f.read()
            # Append report to previous reports by PHN
            all_files[root]["content"] += content
            all_files[root]["files"].append(sorted_file_path)

# Change all keywords to lowercase
cancer_keywords = {
    cancer_type: [keyword.lower() for keyword in keywords]
    for cancer_type, keywords in cancer_keywords.items()
}

# Empty dictionary to track keyword counts per PHN
folder_keyword_counts = {}

# Count keywords in the combined content for each folder
for folder, folder_data in all_files.items():
    combined_content = folder_data["content"].lower()  # All reports per PHN, ensure lowercase
    
    # Initialize keyword counts for this folder
    folder_keyword_counts[folder] = {cancer_type: 0 for cancer_type in cancer_keywords.keys()}
    
  # Count keywords in the combined content using fuzzy matching
    for cancer_type, keywords in cancer_keywords.items():
        for keyword in keywords:
            similarity = fuzz.partial_ratio(keyword.lower(), combined_content.lower())
            if similarity >= 85:
                folder_keyword_counts[folder][cancer_type] += 1


# Print the first 25 keyword counts for each PHN to ensure it works
i = 0
for folder, keyword_count in folder_keyword_counts.items():
    if i < 25:
        print(f"Directory: {folder}")
        for cancer_type, count in keyword_count.items():
            print(f"  {cancer_type}: {count} keyword(s) found")
        i = i + 1
        print("\n")  # Add a newline for better readability between folders


Directory: ./Sorted_by_PHN_Cleaned_Reports\9000826150
  breast_cancer: 0 keyword(s) found
  lung_cancer: 1 keyword(s) found
  lymph_cancer: 0 keyword(s) found
  brain_cancer: 0 keyword(s) found
  skin_cancer: 1 keyword(s) found
  colon_cancer: 0 keyword(s) found
  uterine_or_ovarian_cancer: 0 keyword(s) found
  vulvar_cancer: 0 keyword(s) found
  leukemia_cancer: 0 keyword(s) found
  cervical_cancer: 0 keyword(s) found
  stomach_cancer: 0 keyword(s) found
  pancreatic_cancer: 0 keyword(s) found
  kidney_cancer: 7 keyword(s) found
  thyroid_cancer: 0 keyword(s) found
  liver_cancer: 0 keyword(s) found
  bladder_cancer: 1 keyword(s) found


Directory: ./Sorted_by_PHN_Cleaned_Reports\9001641604
  breast_cancer: 0 keyword(s) found
  lung_cancer: 0 keyword(s) found
  lymph_cancer: 0 keyword(s) found
  brain_cancer: 0 keyword(s) found
  skin_cancer: 0 keyword(s) found
  colon_cancer: 0 keyword(s) found
  uterine_or_ovarian_cancer: 0 keyword(s) found
  vulvar_cancer: 0 keyword(s) found
  leuk

In [84]:
# Path for sorted files
sorted_path = "../Sorted_By_Cancer_Type"
# Negation keywords to rechecking the unsorted items
negation_words = ["negative", "not", "no", "none", "free from", "without", "does not", "not have"]

# Remove existing files and directory if they exist
if os.path.exists(sorted_path):  # Check if the directory exists
    shutil.rmtree(sorted_path)  # Delete its contents if it does
    print("Reports Deleted")

def word_distance_checker(keyword, text, words, distance): #Checks if a word is found within a char distance of a keyword
                for word in words:
                    index = text.find(word)
                    while index != -1:
                        if abs(text.find(keyword) - index) <= distance:
                            return True
                        index = text.find(word, index + 1)
                return False
    
    
# Check that the output path exists
os.makedirs(sorted_path, exist_ok=True)

# Portion to remove from the original paths
uneeded_path = "../Sorted_by_PHN_Cleaned_Reports"

for folder, keyword_count in folder_keyword_counts.items():
    for file_path in all_files[folder]["files"]:
        cleaned_file_name = file_path.replace(uneeded_path, "").lstrip("\\/")

        # Find the cancer type with the highest count
        max_count = max(keyword_count.values())
        max_cancer_types = [cancer_type for cancer_type, count in keyword_count.items() if count == max_count]

        # Determine the target directory
        if max_count == 0:
            target_dir = os.path.join(sorted_path, "unknown_type") #Unknown reports where no keywords at all match
        elif len(max_cancer_types) > 1:  # If there are more than one cancer type with the same count, check for negation keywords
            with open(file_path, "r") as f:
                content = f.read().lower() #Read the file again  

            adjusted_keyword_counts = keyword_count.copy() #Reuse the old keyword count for checkig
            for cancer_type in max_cancer_types:
                for keyword in cancer_keywords[cancer_type]:
                    if word_distance_checker(keyword, content, negation_words, 150): #Check if any negation keywords fall within 150 char of a keyword
                        adjusted_keyword_counts[cancer_type] -= 1 # If so decrement that value by 1

            max_count = max(adjusted_keyword_counts.values()) # Update max count
            max_cancer_types = [cancer_type for cancer_type, count in adjusted_keyword_counts.items() if count == max_count]

            if len(max_cancer_types) > 1: # Add them to conflicting types if its less than 5 cancer types
                #Helps for fixing in the future
                target_dir = os.path.join(sorted_path, "conflicting_cancer_types") 
                #print("Filename", cleaned_file_name, "Still conflicting:", max_cancer_types) #Print the types and file name for testing
            else:
                target_dir = os.path.join(sorted_path, max_cancer_types[0]) #Else save them to their new corrected directory
        else:
            target_dir = os.path.join(sorted_path, max_cancer_types[0]) #Original else to save them if the max value was unique

        # Ensure the target directory exists
        os.makedirs(target_dir, exist_ok=True)

        # Make the new folder path
        original_folder = os.path.dirname(cleaned_file_name)
        new_folder = os.path.join(target_dir, original_folder)
        os.makedirs(new_folder, exist_ok=True)

        # Save the Cancer sorted files to the new directory
        with open(file_path, "r") as f:
            content = f.read()
        new_file_path = os.path.join(new_folder, os.path.basename(cleaned_file_name))
        with open(new_file_path, "w") as f:
            f.write(content)

print("Done")


Reports Deleted
Done


In [8]:
# Ensure all files and folder names are unique within this directory, i.e. no duplicates

def calculate_file_hash(file_path):
    """Calculate the SHA-256 hash of a file."""
    hasher = hashlib.sha256()
    with open(file_path, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hasher.update(chunk)
    return hasher.hexdigest()

def check_unique_files(directory_path):
    """Check if all files in a directory have unique contents."""
    file_hashes = set()
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):  # Skip directories
            file_hash = calculate_file_hash(file_path)
            if file_hash in file_hashes:
                print(f"Duplicate file found: {file_name}")
                return False
            file_hashes.add(file_hash)
    print("All files are unique.")
    return True

def check_unique_folders(directory_path):
    """Check if all folder names in a directory are unique."""
    folder_names = set()
    for item in os.listdir(directory_path):
        item_path = os.path.join(directory_path, item)
        if os.path.isdir(item_path):  # Check only directories
            if item in folder_names:
                print(f"Duplicate folder found: {item}")
                return False
            folder_names.add(item)
    print("All folder names are unique.")
    return True

# Specify your directory path
sorted_path = "../Sorted_By_Cancer_Type"

# Perform checks
files_are_unique = check_unique_files(sorted_path)
folders_are_unique = check_unique_folders(sorted_path)

if files_are_unique and folders_are_unique:
    print("All files and folder names are unique.")
else:
    print("Duplicates found.")



All files are unique.
All folder names are unique.
All files and folder names are unique.
